# Thông tin nhóm 14

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# 1. Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Tình hình tội phạm.
- ...

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# 2. Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# 3. Thu thập dữ liệu

## 3.1. Parse HTML

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại  California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

Thời gian lấy dữ liệu: Ngày 10/12/2019

In [ ]:
# Import các thư viện cần thiết.
import urllib.robotparser
import json
import os.path
import re

from requests_html import HTMLSession

In [ ]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()

In [ ]:
# Lấy link chứa danh sách các ngôi nhà đã bán theo Quận tại California
check_parse = rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

base_url = "https://m.realtytrac.com"
list_url_county = []

# Kiểm tra việc parse HTML được cho phép hay không?
if check_parse == True:
    session = HTMLSession()
    r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

    counties = r.html.find("option")

    for county in counties:
        tail_url = county.attrs['value']
        if (tail_url):
            url = base_url + tail_url
            list_url_county.append(url)


list_url_county

In [ ]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt.
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1


# Hàm parseDetailPage: parse để lấy thông tin chi tiết của từng căn nhà từ detailUrl (của căn nhà đã bán).
# Tham số: session và detailUrl của căn nhà cần parse.
# Trả về: dictionary chứa các thông tin đã parse được.
def parseDetailPage(session: HTMLSession, detailUrl: str):

    # Lấy mã HTML của trang web.
    r = session.get(detailUrl)

    # Dictionary lưu lại kết quả.
    result = {}

    # Lấy address của estate.
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text

    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text

    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text

    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text

    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code

    result["address"] = address

    # Lấy thông tin ngày đã bán căn nhà.
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None

    # Lấy giá cho thuê trước khi bán.
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None


    # Lấy phân mô tả chi tiết về căn nhà.
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)  # sở dĩ lấy mà HTML để dễ parse sau này (khi muốn lấy thông tin từ details)
    result["details"] = details

    # Lấy property info của căn nhà.
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    # Lấy property taxes của căn nhà.
    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes

    # Lấy số lượng trường học gần đó.
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))

    result["school"] = num_school

    # Lấy thông tin tội phạm trong vùng.
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value

    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó.
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc

    return result

In [ ]:
# Hàm parseSearchPage: parse theo page (mỗi parse chứa danh sách và url của 25 căn nhà đã được bán).
# Tham số: session và url của page hiện tại.
# Trả về: list thông tin chi tiết của tất cả căn nhà tại page hiện tại và url của page kế tiếp.
def parseSearchPage(session: HTMLSession, urlPage: str):

    # Lấy mã HTML.
    r = session.get(urlPage)

    # Lấu detailUrl của tất cả bất động sản.
    detailUrls = [link for link in r.html.absolute_links if 'property' in link]

    results = []  # List chứa kết quả parse được tại page.

    # Tiến hành parse từ detailUrl
    for detailUrl in detailUrls:
        # Kiểm tra việc parse HTML được cho phép hay không?
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue

        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)

    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [ ]:
# Hàm getAllCountyProperties: parse tất cả các page của một County để lấy thông tin chi tiết của 
# căn nhà được bán thành công.
# Tham số: session và countryUrl chứa thông tin các căn nhà được bán của một County.
# Trả về: File chứa kết quả đã parse thành công (tên file là tên County được parse).
def getAllCountyProperties(session: HTMLSession, countyUrl: str):
    # Lấy tên County từ urlCounty.
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"

    # Kiểm tra sự tồn tại file.
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return

    # Mở file để ghi dữ liệu (kiểu file là json)
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 1):
            # Kiểm tra việc parse HTML được cho phép hay không?
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue

            # Tiến hành parse theo từng page.
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)

            # Ghi kết quả xuống file.
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")

            # Kiểm tra có còn page tiếp theo hay không?
            if not nextPage:
                break

            # Mỗi lần parse 1 page thì cho sleep 2s.
            time.sleep(2)

Tiến hành lấy dữ liệu. Lưu ý trong quá trình lấy có thể bị timeout do đường truyền mạng, nếu bị thì chỉ cần làm theo các bước sau:
- Xem đã lấy tới page mấy. vd: "...page-125"
- Đổi lại tên file trong thư mục data là "orange-county-1.json" chẳng hạn (số 1, 2, 3,.. là theo mỗi lần lấy dữ liệu của mình).
- Vào hàm "getAllCountyProperties" đổi lại dòng "for url in urlGenerator(countyUrl, 1)" thành "for url in urlGenerator(countyUrl, 125)", số 125 là tùy mình lấy tới page thứ mấy mà bị timeout.
- Dữ liệu tất nhiên sẽ bi trùng, cứ việc yên tâm vì phần sau sẽ xóa trung.

In [ ]:
url = "https://m.realtytrac.com/mapsearch/sold/ca/orange-county"
session = HTMLSession()
getAllCountyProperties(session, url)

In [ ]:
# Phần gộp từng phần dữ liệu đã lấy dang dở lại thành 1 file.
# Nếu không bị timeout như trường hợp ở trên thì không dùng shell code này.
inputFiles = ["data/orange-county-1.json", "data/orange-county-2.json", 
              "data/orange-county-3.json", "data/orange-county-4.json"]
outputFile = "data/full-orange-county.json"

with open(outputFile, "a+") as fOut:
    for inputFile in inputFiles:
        with open(inputFile, "r") as fIn:
            data = fIn.read()
            fOut.write(data)

In [ ]:
# Phần convert file json sang file csv.

def xstr(s):
    return '' if s is None else str(s)


# Hàm preprocessFile: chuyển file json sang file csv.
# Tham số: tên file json và tên file csv.
# Không trả về.
def preprocessFile(inputFile: str, outputFile: str):

    # Kiểm tra tồn tại file Input.
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return

    # Kiểm tra tồn tại file Output.
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return

    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )

        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t".
        fOut.write(f"{tab.join(headers)}\n")

        # Duyệt từng line trong file Input (json).
        for line in fIn:
            row = json.loads(line)

            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")

            # Lấy ngày bán thành công.
            date_sold = row.get("date_sold")

            # Lấy giá cho thuê mỗi tháng (trước khi được bán).
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))

            # Lấy phần info
            info = row.get("info")

            # Type.
            info_type = info.get("type")

            # Bedrooms.
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)

            # Bathrooms.
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)

            # Size.
            info_size = info.get("size")
            if info_size == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",", "").replace(" sqft", ""))

            # Lot size.
            info_lot_size = info.get("lot_size")
            if info_lot_size == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",", "").replace(" sqft", "").replace(" acres", ""))

            # Year build.
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)

            # Est value.
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$", "").replace(",",""))

            # Sold price.
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$", "").replace(",", ""))

            # ID.
            info_property_id = info.get("property_id")

            # County.
            info_county = info.get("county")

            # Parcel_number.
            info_parcel_number = info.get("parcel_number")

            # Lấy phần Taxes.
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$", "").replace(",", ""))
                taxes_improvements = float(taxes.get("improvements").replace("$", "").replace(",", ""))
                taxes_total = float(taxes.get("total").replace("$", "").replace(",", ""))
                taxes_taxes = taxes.get("taxes").replace("$", "").replace(",", "")

            # Lấy số lượng scholl gần đó.
            school = int(row.get("school"))

            # Lấy tình hình tội phạm.
            crime = row.get("local_crime_index")
            if crime is not None:
                total_crime = crime.get("total_crime")
                if total_crime is not None:
                    total_crime = int(total_crime)
                violent_crime = crime.get("violent_crime")
                if violent_crime is not None:
                    violent_crime = int(violent_crime)
                property_crime = crime.get("property_crime")
                if property_crime is not None:
                    property_crime = int(property_crime)

            # Lấy số lượng foreclosures gần đó.
            foreclosures = int(row.get("foreclosures"))

            # Row này là tổng hợp các thông tin lấy được ở trên.
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )

            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

&#9889; Sở dĩ nhóm không thêm phần `description` vào dữ liệu là bởi vì khi quan sát phần mã `html` của `description` thì phát hiện ra nó được viết theo một `format` nhất định, các thông tin số liệu của nó được lấy từ các bảng số liệu trong cùng một trang web (tức là các phần thông tin nhà, thuế,.. mà mình đã parse ở trên). Nếu số liệu nào có thì sẽ được lấy ra và tạo một câu mô tả theo `format` cho trước, còn số liệu nào không có thì hiển nhiên không xuất hiện câu đó. Như vậy, có thể kết luận là phần `description` lúc này sẽ không giúp ích gì cho chúng ta.

In [ ]:
inputFile = "data/full-orange-county.json"
outputFile = "data/orange-county.csv"
preprocessFile(inputFile, outputFile)

## 3.2. Lựa chọn dữ liệu của năm 2019 và có "correct ouput"

In [ ]:
# Import các thư viện cần thiết.
import pandas as pd

### 3.2.1 Xóa dữ liệu trùng

#### Dữ liệu ban đầu

In [ ]:
# Đọc dữ liệu lên
file_csv = "data/Orange-County/orange-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

In [ ]:
data_df.info()

Dữ liệu ban đầu có 21268 dòng.

#### Dữ liệu sau khi xóa trùng

In [ ]:
# Xóa các phần dữ liệu bị trùng
unique_data_df = data_df.drop_duplicates()

In [ ]:
# Xem phần dữ liệu khi đã xóa trùng
unique_data_df.info()

Dữ liệu sau khi xóa trùng còn lại 18893 dòng.

In [ ]:
# Ghi dữ liệu xuống file csv
unique_data_df.to_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t', index=False)

### 3.2.2. Lấy dữ liệu của năm 2019

In [ ]:
# Đọc dữ liệu lên
df = pd.read_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t')
df.head(10)

In [ ]:
df.info()

In [ ]:
# Thêm cột năm bán vào dữ liệu
df = df.assign(year_sold=df.date_sold.str[-4:])

In [ ]:
# Xem dữ liệu này thu thập của các năm nào
df.year_sold.unique()

In [ ]:
# Chỉ lấy phần dữ liệu của năm 2019
df = df[df.year_sold == "2019"]
df.year_sold = df.year_sold.astype(int)

In [ ]:
# Ghi dữ liệu xuống file csv
df.to_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t', index=False)

### 3.2.3. Lấy dữ liệu của năm 2019 có "correct output"

In [ ]:
import pandas as pd

In [ ]:
# Đọc dữ liệu
data_df = pd.read_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t')
data_df.head(10)

In [ ]:
data_df.info()

Dữ liệu có 16011 căn nhà được bán trong 2019

Ta chỉ lấy dữ liệu có `info_sold_price` khác `null`. Như đã nói ở phần cuối của `1.1. Parse HTML` thì không thể tìm kiếm các giá trị chính xác từ phần `decription` để có thể lắp đầy giá trị `null`, mà nếu có tìm cũng chỉ toàn ra giá trị trung bình của các căn nhà được bán lân cận chứ không phải giá trị chính xác của nó.

In [ ]:
df = data_df[pd.notna(data_df["info_sold_price"])]

In [ ]:
df.info()

Dữ liệu còn lại là 15755.

&#9889; Sau khi xóa các dòng có sold_price là `null`. Xem lại cột `info_est_value` thấy toàn `null`, cột này ý nghĩa là giá trị định giá của căn nhà trên các bản rao bán (chưa chốt giá). Nên xóa đi cột này là hoàn toàn hợp lý.

In [ ]:
df = df.drop(columns=["info_est_value"])
df.info()

Ta cần kiểm tra xem ứng với `15755` dòng dữ liệu thì phải có đúng `15755` `info_property_id` hay không?

In [ ]:
len(df.info_property_id.unique())

OK ổn vì có đúng `15755` `info_property_id` như mong muốn

Bây giờ, ta kiểm tra liệu ứng với mỗi `address_street` liệu chỉ có duy nhất một căn nhà được bán thành công hay không?

In [ ]:
address_street_count = df.address_street.value_counts()
address_street_count

Như vậy có `38` `address_street` có số lượng nhà bán thành công trong năm 2019 lơn hơn 1, trong đó lại có `1600 S Disneyland Dr` có số lần xuất hiện lên đến con sô `16`, điều này hơi kỳ lạ. Vì vậy, ta cần quan sát thêm về các `addresss_street` kiểu này.

In [ ]:
# Lấy tên các addresss_street có số lượng nhà được bán trong năm 2019 nhiều hơn 1
duplicate_addresss_street = address_street_count[address_street_count > 1].index
duplicate_addresss_street

In [ ]:
len(duplicate_addresss_street)

Xem thử tất cả các dòng có `address_street` là `1600 S Disneyland Dr` (hay `duplicate_addresss_street[0]`)

In [ ]:
tmp_df = df[df.address_street == duplicate_addresss_street[0]]
tmp_df

In [ ]:
tmp_df.info_sold_price

Đối với những dòng có cùng `address_street` mà lại khác `date_sold` và `info_sold_price` thì nên bỏ là hợp lý và khó có thể lấy được chính xác giá trị `sold_price`.

Tiếp tục, xem thử tất cả các dòng có `address_street` là `200 W Midway Dr ` (hay `duplicate_addresss_street[1]`)

In [ ]:
tmp_df = df[df.address_street == duplicate_addresss_street[1]]
tmp_df

In [ ]:
tmp_df.info_sold_price

Mặc dù có cùng `address_street`, cùng `date_sold` và cùng `info_sold_price` nhưng lại có các giá trị `taxes` thay đổi một cách kỳ lại + với các thông tin chính (`info_bedrooms`, `info_bathroooms`,...) bị thiếu. Phần các thông tin chính này có thể giải quyết bằng `fill miss value` (`mean`, `median`, `most`,...) nhưng phần `taxes` thì nhóm không biết xác định như thế nào cho hợp lý và gần chính xác nhất. Nên quyết định bỏ luôn các dòng dạng này.

In [ ]:
sum(address_street_count[address_street_count > 1])

Có `106` dòng dữ liệu bị như thế, so với `15755` dòng dữ liệu hiện có thì chúng chưa chiếm đến `1%` nên nhóm quyết định bỏ đi là hợp lý.

In [ ]:
df = df[~df.address_street.isin(duplicate_addresss_street)]
df.info()

In [ ]:
# Ghi dữ liệu xuống file csv.
df.to_csv("data/Orange-County/data-2019.csv", sep='\t', index=False)

#### Tách dữ liệu thành 2 tập train và test theo tỉ lệ 80% 20%

In [ ]:
# Đọc dữ liệu lên
data_df = pd.read_csv("data/Orange-County/data-2019.csv", sep='\t')
data_df.head()

In [ ]:
data_df.info()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Đổi tên
data_df = data_df.rename(columns={"info_sold_price": "sold_price"})

In [ ]:
# Tách dữ liệu theo tỉ lệ 80% và 20%
train, test = train_test_split(data_df, test_size=0.2, random_state=0)
train.head().index

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train.head().index` của nhóm ra 5 giá trị là: [2746, 8859, 11288, 2153, 12119]. 

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# Ghi tập train
train.to_csv("data/Orange-County/train-data-2019.csv", sep='\t', index=False)

In [ ]:
# Ghi tập test
test.to_csv("data/Orange-County/test-data-2019.csv", sep='\t', index=False)

# 4. Tiền xử lý dữ liệu

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [160]:
data_df = pd.read_csv("data/Orange-County/train-data-2019.csv", sep='\t')
data_df.head()

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,13242 Amarillo Dr,Westminster,CA,92683,10/24/2019,3157.0,Single Family Residence,3.0,1.0,1090.0,...,508260.0,54780.0,563040.0,6969 (1.23 %),3,97.0,79.0,140.0,4,2019
1,716 S Palomino Ln,Anaheim,CA,92807,08/16/2019,2748.0,Single Family Residence,3.0,3.0,1582.0,...,371674.0,148526.0,520200.0,5527 (1.06 %),3,88.0,71.0,127.0,4,2019
2,19702 Lancewood Plz,Yorba Linda,CA,92886,07/23/2019,2966.0,Single Family Residence,4.0,2.0,1440.0,...,29459.0,43373.0,72832.0,1240 (1.70 %),3,22.0,12.0,44.0,4,2019
3,25885 Trabuco Rd Apt 59,Lake Forest,CA,92630,10/30/2019,1726.0,Condominium,2.0,2.0,1190.0,...,112879.0,93656.0,206535.0,2096 (1.01 %),3,29.0,23.0,42.0,4,2019
4,6742 Gate Hill Cir,Huntington Beach,CA,92648,07/15/2019,5860.0,Single Family Residence,4.0,3.0,2193.0,...,159764.0,148957.0,308721.0,3706 (1.20 %),3,57.0,47.0,82.0,4,2019


In [161]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12519 entries, 0 to 12518
Data columns (total 26 columns):
address_street        12485 non-null object
address_locality      12519 non-null object
address_region        12519 non-null object
address_code          12519 non-null int64
date_sold             12519 non-null object
mortgage              12519 non-null float64
info_type             12519 non-null object
info_bedrooms         11363 non-null float64
info_bathrooms        11371 non-null float64
info_size             11946 non-null float64
info_lot_size         8484 non-null float64
info_year_built       11571 non-null float64
sold_price            12519 non-null float64
info_property_id      12519 non-null int64
info_county           12519 non-null object
info_parcel_number    12519 non-null int64
taxes_land            12321 non-null float64
taxes_improvements    12321 non-null float64
taxes_total           12321 non-null float64
taxes_taxes           12321 non-null object
schoo

Tập train có `12519` dòng dữ liệu và `26` cột.

In [162]:
data_df.describe(include="all")

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
count,12485,12519,12519,12519.000000,12519,1.251900e+04,12519,11363.000000,11371.000000,11946.000000,...,1.232100e+04,1.232100e+04,1.232100e+04,12321,12519.000000,12156.000000,12156.000000,12156.000000,12519.000000,12519.0
unique,12485,52,2,NaN,155,NaN,14,NaN,NaN,NaN,...,NaN,NaN,NaN,11959,NaN,NaN,NaN,NaN,NaN,NaN
top,24621 Harbor View Dr Unit B,Irvine,CA,NaN,06/28/2019,NaN,Single Family Residence,NaN,NaN,NaN,...,NaN,NaN,NaN,10555 (1.12 %),NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1107,12517,NaN,232,NaN,8162,NaN,NaN,NaN,...,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,92552.015816,NaN,8.928697e+03,NaN,3.124263,2.602102,2426.583208,...,5.009780e+05,2.826592e+05,7.836531e+05,NaN,2.884016,54.516946,42.570171,82.203274,3.853822,2019.0
std,NaN,NaN,NaN,564.557227,NaN,7.526277e+04,NaN,1.008088,0.991297,7797.643516,...,1.434805e+06,1.751883e+06,2.832966e+06,NaN,0.578382,29.778698,28.960716,37.985382,0.750273,0.0
min,NaN,NaN,NaN,90620.000000,NaN,2.000000e+00,NaN,1.000000,1.000000,301.000000,...,4.410000e+02,0.000000e+00,0.000000e+00,NaN,0.000000,13.000000,1.000000,23.000000,0.000000,2019.0
25%,NaN,NaN,NaN,92630.000000,NaN,2.548000e+03,NaN,2.000000,2.000000,1264.000000,...,1.504430e+05,7.841300e+04,2.769060e+05,NaN,3.000000,29.000000,22.000000,48.000000,4.000000,2019.0
50%,NaN,NaN,NaN,92677.000000,NaN,3.339000e+03,NaN,3.000000,3.000000,1671.000000,...,3.111640e+05,1.312930e+05,4.712400e+05,NaN,3.000000,55.000000,39.000000,82.000000,4.000000,2019.0
75%,NaN,NaN,NaN,92804.000000,NaN,4.610000e+03,NaN,4.000000,3.000000,2377.750000,...,5.119080e+05,2.319820e+05,7.320000e+05,NaN,3.000000,71.000000,54.000000,122.000000,4.000000,2019.0


## 4.1. Tách tập dữ liệu thành 2 phần train (70%) và validation (30%)

In [163]:
# Tách X và y
y_sr = data_df["sold_price"] # sr là viết tắt của series
X_df = data_df.drop("sold_price", axis=1)

In [164]:
# Tách tập train và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)
train_X_df.head().index

Int64Index([10747, 12214, 5916, 489, 12257], dtype='int64')

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train_X_df.head().index` của nhóm ra 5 giá trị là: [10747, 12214, 5916, 489, 12257]. 

In [165]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 10747 to 2732
Data columns (total 25 columns):
address_street        8736 non-null object
address_locality      8763 non-null object
address_region        8763 non-null object
address_code          8763 non-null int64
date_sold             8763 non-null object
mortgage              8763 non-null float64
info_type             8763 non-null object
info_bedrooms         7951 non-null float64
info_bathrooms        7956 non-null float64
info_size             8361 non-null float64
info_lot_size         5931 non-null float64
info_year_built       8099 non-null float64
info_property_id      8763 non-null int64
info_county           8763 non-null object
info_parcel_number    8763 non-null int64
taxes_land            8632 non-null float64
taxes_improvements    8632 non-null float64
taxes_total           8632 non-null float64
taxes_taxes           8632 non-null object
school                8763 non-null int64
total_crime           84

`train_X_df` có `8763` dòng dữ liệu và `25` cột.

In [166]:
val_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 1540 to 4911
Data columns (total 25 columns):
address_street        3749 non-null object
address_locality      3756 non-null object
address_region        3756 non-null object
address_code          3756 non-null int64
date_sold             3756 non-null object
mortgage              3756 non-null float64
info_type             3756 non-null object
info_bedrooms         3412 non-null float64
info_bathrooms        3415 non-null float64
info_size             3585 non-null float64
info_lot_size         2553 non-null float64
info_year_built       3472 non-null float64
info_property_id      3756 non-null int64
info_county           3756 non-null object
info_parcel_number    3756 non-null int64
taxes_land            3689 non-null float64
taxes_improvements    3689 non-null float64
taxes_total           3689 non-null float64
taxes_taxes           3689 non-null object
school                3756 non-null int64
total_crime           365

`val_X_df` có `3756` dòng dữ liệu và `25` cột.

## 4.2. Quan sát và lựa chọn dữ liệu bằng kiến thức cá nhân trên tập train (`train_X_df` và `train_y_sr`)

### 4.2.1. Quan sát cột `address_street`

In [ ]:
# https://stackoverflow.com/questions/55105280/how-to-extract-apartment-from-address-in-pandas
pat = r'^(?P<number>\d+)?(?P<direction>.\w(?= ))?(?P<street>.+(?=\bapt|\bunit)|.+(?=#)|.+)(?P<apt_unit>(?:\bapt|\bunit|#|).+)?'
lower_address_street = train_X_df.address_street.str.lower()
tmp = lower_address_street.str.extract(pat)
tmp

In [ ]:
train_X_df.iloc[10]

In [ ]:
tmp["street"].value_counts()

In [ ]:
sum(tmp["street"].value_counts() > 1)

In [ ]:
len(tmp["street"].unique())

Theo kết quả tìm kiếm trên internet (https://cartographic.info/usa/street/show.php?p=california&t=Orange%20County) thì có `280` tên đường lớn tại `Orange County`. Vậy trong danh sách mà nhóm parse được có đến `6051` tên đường khác nhau, chắc bao gồm những con đường nhỏ. Việc dùng `street` cho huấn luyện mô hình sẽ không hiệu quả, vì khi đưa vào mô hình bằng phương pháp one-hot chắc chắn sẽ tạo ra số chiều rất lớn và với một số địa chỉ xuất hiện rất ít dễ dẫn đén overfitting.

In [ ]:
len(tmp[pd.notna(tmp["apt_unit"])])

Nhóm sẽ xóa cột `address_street` và thêm cột `apt_unit` được rút trích ở trên vào dữ liệu ở bước tiền xử lý dữ liệu, cột này chỉ có 2 giá trị (`True`/`False`)

### 4.2.2. Quan sát các cột `address_locality`, `address_region` và `address_code`

&#9889; Theo tìm hiểu của nhóm, `address_code` là tổng hợp từ `address_locality` và `address_region`

In [9]:
unique_code = train_X_df.address_code.unique()
len(unique_code)

87

In [10]:
train_X_df.address_code.value_counts()

92677    286
92630    224
92694    218
92688    206
90631    201
92886    200
92618    199
92656    193
92692    192
92646    183
92620    177
92691    171
92679    157
92780    153
92833    152
92648    150
92627    150
92660    146
92672    143
92805    141
92804    140
92649    137
92705    137
92869    136
92629    133
92673    131
92807    131
92870    127
92683    126
92708    126
        ... 
92625     67
90720     66
92802     65
92706     65
90621     63
90680     63
92603     60
92604     59
92843     58
92841     57
92845     53
92703     52
92657     49
90740     48
92606     44
92832     43
92868     43
92844     41
92866     38
90623     32
92624     32
92676     27
92661     21
92861     20
92823     19
92662     16
92655      8
92617      6
90742      4
92685      1
Name: address_code, Length: 87, dtype: int64

In [ ]:
for code in unique_code:
    tmp_df = train_X_df[train_X_df.address_code == code]
    print("Code: " + str(code))
    print("\n + Locality: " + str(tmp_df.address_locality.unique()))
    print("\n + Region: " + str(tmp_df.address_region.unique()))
    print("\n")

Theo nhóm tìm hiểu thì mã `code` (`address_code`) kia là mã bưu điện, mã này có thể dùng cho nhiều `locality` ở gần nhau, do đó ta kết quả như ở trên là hoàn toàn bình thường. Giờ nhóm quyết định bỏ 2 cột `address_locality` và `addres_region`, chỉ giữ lại cột `address_code` là đủ.

2. Xem thử cột `address_code` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_code = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.address_code)])
print("The number of missing values (column 'address_code'): " + str(num_miss_code))

Good. Cột `address_code` không có giá trị thiếu.

### 4.2.3. Quan sát cột `date_sold`

Đây là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)

1. Xem thử cột `date_sold` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_date_sold = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.date_sold)])
print("The number of missing values (column 'date_sold'): " + str(num_miss_date_sold))

Good. Cột `date_sold` không có giá trị thiếu.

2. Rút trích thông tin từ cột `date_sold`

In [ ]:
train_X_df.date_sold.unique()

Cột `date_sold` được ghi dưới dạng `MM/DD/YYYY`. Giờ nhóm sẽ rút trích `Month` từ `date_sold`.

In [12]:
month = train_X_df.date_sold.str[:2].astype(int)
sorted(month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Sở dĩ không có tháng `12` là do thời gian lấy dữ liệu của nhóm là đầu tháng `12` nên trang web chưa cập nhật. Về sau ở bước `Xây dựng pipline tiền xử lý dữ liệu`, nhóm sẽ tiến hành xóa cột `date_sold`, thêm cột `Month` và muốn thử để nguyên `11` tháng huấn luyện tốt hơn hay chia ra thành `4` mùa tốt hơn bằng cách đặt thêm cờ hiệu `month_to_seasion`.

In [13]:
# Cách chia month theo 4 mùa
season = (month%12 + 3)//3
season.value_counts()

3    4709
4    4017
2      28
1       9
Name: date_sold, dtype: int64

### 4.2.4. Quan sát cột `mortgage`

Đây là cột dạng số (numerical) - float64

`mortgage` là tiền cho thuê tính theo tháng của căn nhà khi chưa được bán, nó cũng gần gần để suy ra `sold_price` rồi.

In [ ]:
train_X_df.mortgage.describe()

Xem thử cột `mortgage` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_mortgage = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.mortgage)])
print("The number of missing values (column 'mortgage'): " + str(num_miss_mortgage))

Good. Cột `mortgage` không có giá trị thiếu.

Nhưng, nhóm lại phát hiện khi lấy `mortgage` nhân cho `220` sẽ ra con số rất gần với giá tiền của căn nhà, nhưng vậy phải xóa `mortgage` đi thì mới áp dụng các mô hình máy học để dự đoán giá nhà được.

### 4.2.5. Quan sát cột `info_type`

Đây là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)

1. Quan sát các giá trị xuất hiện trong cột và số lượng tương ứng

In [123]:
len(train_X_df.info_type.unique())

13

In [167]:
train_X_df.info_type.value_counts()

Single Family Residence                            5709
Condominium                                        2399
MISCELLANEOUS                                       326
Multi-Family Dwellings                              230
Contact Agent                                        70
Miscellaneous Structures - Ranch, Farm Fixtures      10
Residential - Vacant Land                             5
Cooperative                                           4
Miscellaneous (Residential)                           3
Governmental / Public Use (general)                   3
Miscellaneous (general)                               2
Agricultural (unimproved) - Vacant Land               1
Duplex (2 units, any combination)                     1
Name: info_type, dtype: int64

Có một số giá trị xuất hiện rất ít, nhóm thử dùng thêm tham số `num_top_type` để chọn ra các `info_type` xuất hiện nhiều lần; còn các giá trị nằm ngoài `num_top_type` nhóm sẽ cho là giá trị `Others`. (như `BT03-TienXuLy_ChongOverfit`)

2. Xem thử cột `address_type` có bao nhiêu giá trị thiếu

In [ ]:
num_miss_type = train_X_df.shape[0] - len(train_X_df[pd.notna(train_X_df.info_type)])
print("The number of missing values (column 'info_type'): " + str(num_miss_type))

Good. Cột `info_type` không có giá trị thiếu.

### 4.2.5. Quan sát cột `info_bedrooms` và cột `info_bathrooms`

Đây là cột dạng số (numerical) - float64

1. Xem thử cột `info_bedrooms` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_bedrooms = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_bedrooms)])
print("The number of missing values (column 'info_bedrooms'): " + str(num_miss_bedrooms))

Cột `info_bedrooms` có `812` giá trị bị thiếu.

In [ ]:
train_X_df.info_bedrooms.value_counts()

In [ ]:
train_X_df.info_bedrooms.describe()

In [ ]:
train_X_df.info_bedrooms.value_counts().sort_index().plot(kind='bar')

Do là `bedrooms` và các giá trị của nó là số nguyên, nên nhóm dùng `most` để lắp đầy các giá trị thiếu cho cột `info_bedrooms`

2. Xem thử cột `info_bathrooms` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_bathrooms = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_bathrooms)])
print("The number of missing values (column 'info_bathrooms'): " + str(num_miss_bathrooms))

Cột `info_bathrooms` có `807` giá trị bị thiếu.

In [ ]:
train_X_df.info_bathrooms.value_counts()

In [ ]:
train_X_df.info_bathrooms.describe()

In [ ]:
train_X_df.info_bathrooms.value_counts().sort_index().plot(kind='bar')

Tương tự nhóm cùng sẽ dùng `most` để lắp đầy các giá trị thiếu cho cột `info_bathrooms`

### 4.2.6. Quan sát cột `info_size` và cột `info_lot_size`

Đây là cột dạng số (numerical) - float64

1. Xem thử cột `info_size` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_size= train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_size)])
print("The number of missing values (column 'info_size'): " + str(num_miss_size))

Cột `info_size` có `402` giá trị bị thiếu.

In [ ]:
train_X_df.info_size.value_counts()

In [ ]:
train_X_df.info_size.describe()

In [ ]:
len(train_X_df.info_size.unique())

Điền các giá trị thiếu bằng `mean`.

2. Xem thử cột `info_lot_size` có bao nhiêu giá trị thiếu và chọn cách để lắp đầy các giá trị thiếu đó.

In [ ]:
num_miss_lot_size= train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_lot_size)])
print("The number of missing values (column 'info_size'): " + str(num_miss_lot_size))

Cột `info_size` có `2832` giá trị bị thiếu.

In [ ]:
train_X_df.info_lot_size.value_counts()

In [ ]:
train_X_df.info_lot_size.describe()

In [ ]:
len(train_X_df.info_lot_size.unique())

Điền các giá trị thiếu bằng `mean`.

### 4.2.7. Quan sát cột `info_year_built`

Đây là cột dạng số (numerical) - float64

Xem thử cột `info_year_built` có bao nhiêu giá trị thiếu

In [14]:
num_miss_year_built = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.info_year_built)])
print("The number of missing values (column 'info_size'): " + str(num_miss_year_built))

The number of missing values (column 'info_size'): 664


Cột `info_year_built` có `664` giá trị bị thiếu.

In [73]:
train_X_df.info_year_built.unique()

array([1972., 1951.,   nan, 1996., 1963., 1961., 2014., 2009., 1955.,
       1980., 2010., 2005., 1969., 1954., 1971., 1995., 1973., 1975.,
       1977., 1990., 1976., 1965., 1985., 1999., 1964., 1989., 1956.,
       1988., 1993., 2001., 1982., 1979., 2004., 1946., 1991., 1998.,
       1950., 1966., 1984., 1987., 1917., 2007., 1962., 1948., 2008.,
       1959., 1997., 2006., 1960., 1957., 1974., 2003., 1925., 1958.,
       1978., 1952., 1949., 1968., 1986., 1970., 2000., 2002., 1953.,
       2012., 1916., 2013., 1945., 1967., 1920., 1994., 1981., 1933.,
       1928., 1929., 1937., 1983., 1894., 1992., 1947., 2011., 1922.,
       1939., 1904., 1942., 1932., 1935., 1910., 1921., 1941., 1938.,
       1900., 1930., 1927., 1944., 1923., 1926., 1915., 1931., 1907.,
       1936., 1914., 1908., 1924., 1940., 1912., 1899., 1934., 1919.,
       1911., 1918., 1943., 1913.])

In [70]:
train_X_df.info_year_built.value_counts()

1964.0    252
1989.0    250
1963.0    225
1972.0    216
1977.0    210
1973.0    205
1965.0    202
1976.0    199
1990.0    197
1955.0    196
1974.0    196
1971.0    186
1986.0    176
1969.0    172
1980.0    169
1978.0    166
1979.0    165
1983.0    162
1987.0    161
1962.0    161
1956.0    158
1984.0    147
1954.0    144
1959.0    144
1985.0    142
1975.0    138
1968.0    127
1960.0    122
2003.0    119
1981.0    116
         ... 
1937.0      8
1940.0      7
1936.0      7
1921.0      7
1926.0      7
1941.0      6
1939.0      5
1931.0      4
1944.0      4
1915.0      4
1914.0      4
1942.0      4
1932.0      3
1912.0      3
1910.0      3
1933.0      3
1908.0      2
1916.0      2
1917.0      2
1907.0      2
1904.0      2
1934.0      2
1894.0      1
1919.0      1
1943.0      1
1900.0      1
1911.0      1
1918.0      1
1899.0      1
1913.0      1
Name: info_year_built, Length: 111, dtype: int64

In [71]:
train_X_df.info_year_built.describe()

count    8099.000000
mean     1976.009137
std        17.817193
min      1894.000000
25%      1964.000000
50%      1976.000000
75%      1989.000000
max      2014.000000
Name: info_year_built, dtype: float64

Nhóm muốn thử để nguyên các năm mà căn nhà được xây vào huấn luyện là tốt hay chia ra thành khoảng thời gian như: trước 1900, từ 1900 đến 1949, từ 1950 đến 1999, từ 2000 trở về sau. Nhóm đặt cờ hiệu `year_to_period`, nếu là `True` thì chuyển thành các khoảng thời gian để huấn luyện, ngược lại vẫn để nguyên vào huấn luyện.

### 4.2.8. Quan sát các cột `info_property_id`, `info_county` và `info_parcel_number`

- `info_property_id` là cột dạng số (numerical) - int64
- `info_county` là cột dạng chuỗi có giá trị rời rạc không thứ tự (categorical)
- `info_parcel_number` là cột dạng số (numerical) - int64

Bỏ đi các cột này, vì 2 cột `info_property_id` và `info_parcel_number` ứng với mỗi dòng dữ liệu là hoàn toàn khác nhau; còn cột `info_county` thì có giá trị là `Orange County` (do dữ liệu thu thập của Quận Cam).

### 4.2.9. Quan sát các cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`

Các cột này đều là cột dạng số (numerical) - float64

In [ ]:
num_miss_taxes = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.taxes_land)])
print("The number of missing values (column 'taxes'): " + str(num_miss_taxes))

Có `131` giá trị thiếu cho mỗi cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`

Search cùng `address_code` thì lắp đầy các giá trị thiếu bằng `mean` của các `taxes` trong cùng `address_code`

Giữ nguyên các cột `taxes_land`, `taxes_improvements` và bỏ cột `taxes_total`. Giải thích thêm ý nghĩa của các cột này:
- Cột `taxes_land` là tiền cải thuế cải tạo đất mà mình phải đóng cho chính phủ.
- Cột `taxes_improvements` là tiền mà mình phải trả do sử dụng các tiện ích của chính phủ (cắt cỏ, tiền rác, củng cố hàng rào,...)
- Cột `taxes_total` là tổng của 2 cột trên.

Đối với cột `taxes_taxes` thì nhóm sẽ rút trích số tiền phải trả trong cột này ra.

In [ ]:
train_X_df.taxes_taxes

In [ ]:
train_X_df.taxes_taxes.str.extract(r'(\d+)', expand=False)

### 4.2.10. Quan sát cột `school` và cột `foreclosures`

Các cột này đều là cột dạng số (numerical) - int64

Hai cột này không có giá trị thiếu, nên được giữ nguyên để huấn luyện.

In [ ]:
train_X_df.school.value_counts().sort_index().plot(kind='bar')

In [ ]:
train_X_df.foreclosures.value_counts().sort_index().plot(kind='bar')

### 4.2.11. Quan sát cột `total_crime`, `violent_crime` và `property_crime`

Các cột này đều là cột dạng số (numerical) - float64

Giải thích về ý nghĩa của các cột này. Giá trị mà các cột `total_crime`, `violent_crime` và `property_crime` biểu diễn được tính theo % so với tỉ lệ tội phạm quốc gia. Ví dụ `total_crime = 79` thì ở đây tổng số tội phạm là `79%` so với toàn nước (`100%`).

In [ ]:
num_miss_crime = train_X_df.shape[0] - len(train_X_df[pd.notnull(train_X_df.total_crime)])
print("The number of missing values (column 'taxes'): " + str(num_miss_crime))

Có `265` giá trị thiếu cho mỗi cột `total_crime`, `violent_crime` và `property_crime`

In [ ]:
train_X_df.total_crime.value_counts()

Search cùng `address_code` thì lắp đầy các giá trị thiếu bằng `median` của các `crime` trong cùng `address_code`

### 4.2.12. Quan sát `correct input` (`sold_price` của ` train_y_sr`)

Đây là cột dạng số (numerical) - float64

In [ ]:
train_y_sr.describe()

In [ ]:
train_y_sr.sort_values().sort_index().plot(style='.')

## 4.3. Thiết kế pipeline tiền xử lý dữ liệu

1. Cột `address_street`: 
    - Ở đây có 4 thông tin được rút trích (xem  phần `4.2.1. Quan sát cột address_street` ở trên):
         + `number`: số nhà, thông tin này không cần thiết cho việc huấn luyện.
         + `direction`: hướng (đông, tây, nam, bắc) của đường, thông tin này không cần thiết cho việc huấn luyện.
         + `street`: tên đường, theo kết quả parse thì thu được đến 6051 tên đường lớn/nhỏ khác nhau. Nếu đưa tên đường này vào huấn luyện thì có 3 vấn đề: thứ nhất là nếu dùng one-hot để chuyển sang dạng số thì sẽ làm tăng số chiều dữ liệu lên rất nhiều (thêm đến trên 6000 thuộc tính), thứ hai do có khoảng 1500 tên đường chỉ xuất hiện duy nhất một lần trong `train_X_df`, còn các tên đường khác thì cũng có tần số xuất hiện khá thấp (cao nhất chỉ có 16 lần) nên nếu thêm vào huấn luyện có thể dẫn đến overfitting, thứ 3 là nếu như ở tập `validation` hay tập `test` mà có tên đường không nằm trong tập `train` thì cũng khó khăn để giải quyết. Nên nhóm quyết định không lấy thông tin `street` để huấn luyện.
         + `apt_unit`: cho biết căn nhà có thuộc chung cư nào hay không, thông tin này nhóm nghĩ là cần thiết cho việc huấn luyện vì nghĩ giá nhà riêng sẽ có phần nào khác với các căn nhà thuộc các khu chung cư. Vì vậy, sẽ thêm cột `is_apt` vào để huấn luyện, cột này chỉ có 2 giá trị `0`/`1` (nếu thuộc chung cư thì `1`, ngược lại thì `0`).
    - Tóm lại, các việc cần làm:
        + Rút trích ra cột `is_apt` (như mô tả ở trên) và thêm vào dữ liệu.
        + Xóa cột `address_street`.
        + Không cần lắp đầy giá trị thiếu, mặc định thiếu là 0 (tức tìm không thấy chung cư trong địa chỉ).
        + Đây là dữ liệu số, để nguyên vậy vào huấn luyện.
         
2. Xóa 2 cột `address_locality` và cột `address_region`, giữ lại cột `address_code` (lý do xem ở phần `4.2.2. Quan sát các cột address_locality, address_region và address_code` ở trên). Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng `most` để lấp đầy. Dùng one-hot để đưa vào huấn luyện.
3. Cột `date_sold`: rút trích `month` ra (xem ở phần `4.2.3. Quan sát cột date_sold`), xóa cột `date_sold` và `year_sol`, thêm vào cột `time_sold` (là giá trị `month` vừa rút trích, đặt tên để tiện cho sau này có chuyển sang `season` hay không thôi). Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng `most` để lấp đầy. Bỏ vào huấn luyện dùng one-hot, nhớ là thử để nguyên `11` tháng huấn luyện tốt hơn hay chia ra thành `4` mùa tốt hơn bằng cách đặt thêm cờ hiệu `month_to_season`.
4. Cột `mortgage`: nếu để nguyên huấn luyện dùng mô hình `Linear Regression` thì cho ra kết quả độ chính xác bất ngờ là `99.99%`, và sau khi thử tính toán một vài records thì nhóm phát hiện ra chỉ cần lấy `mortgage` nhân cho `200` thì ra con số rất gần với giá nhà (`sold_price`), vì có công thức sẵn khi có được số liệu `mortgage`. Như thế thì dự đoán gì nữa, nên nhóm quyết định xóa đi cột `mortgage`.
5. Cột `info_type`: Có một số giá trị xuất hiện rất ít, nhóm thử dùng thêm tham số `num_top_types` để chọn ra các `info_type` xuất hiện nhiều lần; còn các giá trị nằm ngoài `num_top_types` nhóm sẽ cho là giá trị `Others`. (như `BT03-TienXuLy_ChongOverfit`). Ở tập `train_X_df` thì không thấy dữ liệu thiếu, nếu thiếu dùng `most` để lấp đầy. Đưa vào huấn luyện thì dùng one-hot.
6. Hai cột `info_bedrooms` và `info_bathrooms`: lắp đầy giá trị thiều bằng `most`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
7. Hai cột `info_size` và `info_lot_size`: dùng `KNN (n=5)` để lắp đầy giá trị thiếu. Kiểu số, cứ bỏ thẳng vào huấn luyện.
8. Cột `info_year_built` để nguyên vào huấn luyện hoặc chuyển thành các giai đoạn (trước 1900, từ 1900 đên trước 1950, từ 1950 đến trước 2000 và từ 2000 trở về sau) bằng tham số `year_to_period`, lắp đầy giá trị thiếu bằng `most`, đưa vào huấn luyện dùng `one-hot`.
9. Xóa 3 cột` info_property_id`, `info_county` và `info_parcel_number` (lý do xem ở phần `4.2.9. Quan sát các cột info_property_id, info_county và info_parcel_number`).
10. Các cột `taxes_land`, `taxes_improvements`, `taxes_total` và `taxes_taxes`: đầu tiên rút trích `taxes` cho cột `taxes_taxes`, thay bằng dữ liệu tìm được (xem ở phần `4.2.10. Quan sát các cột taxes_land, taxes_improvements và taxes_taxes`), nhớ xóa cột `taxes_total`. Điền các giá trị thiếu bằng `mean`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
11. Giữ nguyên cột `school` và cột `foreclosures`. Nếu có giá trị thiếu thì điền bằng `median`. Kiểu số, cứ bỏ thẳng vào huấn luyện.
12. Các `total_crime`, `violent_crime` và `property_crime`: lắp đầy giá trị thiếu bằng `median`. Kiểu số, cứ bỏ thẳng vào huấn luyện.

- Chuẩn hóa theo cách của thầy trong `BT03`.
- Dùng mô hình `Linear Regression` để huấn luyện mô mình thử xem.

In [20]:
import sklearn
sklearn.__version__

'0.22'

In [181]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
import re

In [102]:
# Class AddressProcessor dùng để xử lý các cột address_street, address_locality, address_region và address_code
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.
class AddressProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Extract các thông tin (số nhà, hướng của đường, tên đường, tên chung cư) từ address_street.
        pat = r'^(?P<number>\d+)?(?P<direction>.\w(?= ))?(?P<street>.+(?=\bapt|\bunit)|.+(?=#)|.+)(?P<apt_unit>(?:\bapt|\bunit|#|).+)?'
        lower_address_street = X_df.address_street.str.lower()
        street_parsed = lower_address_street.str.extract(pat)
        
        # Xóa các cột dùng nữa.
        transformed_df = X_df.drop(["address_street", "address_locality", "address_region"], axis=1)
        
        # Chuyển cột address_code về dạng categorical
        transformed_df["address_code"] = transformed_df["address_code"].astype(str)
            
        # Thêm cột is_apt vào DataFrame (1 là thuộc chung cư, 0 là không không thuộc chung cư).
        transformed_df["is_apt"] = street_parsed["apt_unit"].notnull().astype(int)
        return transformed_df

In [103]:
# TEST AddressProcessor
ap = AddressProcessor()

ap.transform(train_X_df).is_apt.value_counts()

0    7285
1    1478
Name: is_apt, dtype: int64

In [104]:
# Class DateSoldProcessor dùng để xử lý các cột date_sold và year_sold.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class DateSoldProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, month_to_season = True):
        self.month_to_season = month_to_season

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Rút trích month từ date_sold.
        month = X_df.date_sold.str[:2].astype(int)
        
        # Xóa các cột không dùng nữa.
        transformed_df = X_df.drop(["date_sold", "year_sold"], axis=1)
        
        if (self.month_to_season == False):
            # Thêm cột month vào DataFrame (để tên là time_sold cho tiện truy cập).
            transformed_df["time_sold"] = month
        else:
            # 1 là Đông (tháng 12, 1, 2)
            # 2 là Xuân (tháng 3, 4, 5)
            # 3 là Hạ (tháng 6, 7, 8)
            # 4 là Thu (tháng 9, 10, 11)
            season = ((month.astype(int)%12 + 3)//3).astype(str)
            # Thêm cột season_sold vào DataFrame (để tên là time_sold cho tiện truy cập).
            transformed_df["time_sold"] = season
        return transformed_df

In [105]:
# TEST DateSoldProcessor
dsp = DateSoldProcessor(False)

dsp.transform(train_X_df).time_sold.value_counts()

7     1719
8     1707
9     1593
10    1356
6     1283
11    1068
5       12
4        9
3        7
2        5
1        4
Name: time_sold, dtype: int64

In [106]:
# TEST DateSoldProcessor
dsp = DateSoldProcessor(True)

dsp.transform(train_X_df).time_sold.value_counts()

3    4709
4    4017
2      28
1       9
Name: time_sold, dtype: int64

In [182]:
# Class InfoTypeProcessor dùng để xử lý cột info_type.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class InfoTypeProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_types=5):
        self.num_top_types = num_top_types

    def fit(self, X_df, y=None):
        info_type_col = X_df.info_type
        
        # Đếm các dạng type trong cột info_type
        self.type_counts_ = info_type_col.value_counts()
        types = list(map(re.escape, self.type_counts_.index))
        
        # Lấy ra top_type theo tham số num_top_types được truyền vào
        self.top_types_ = types[:max(1, min(self.num_top_types, len(types)))]
        return self

    def transform(self, X_df, y=None):
        # Các cột không phải top_type được mặc định là "Others"
        type_sr = X_df.info_type.str.extract(f'^({"|".join(self.top_types_)})$', expand=False).fillna("Others")
        
        # Xóa đi cột info_type vì không dùng nữa.
        transformed_df = X_df.drop(["info_type"], axis=1)
        
        # Thêm cột type vào DataFrame.
        transformed_df["type"] = type_sr
        return transformed_df

In [188]:
# TEST InfoTypeProcessor
itp = InfoTypeProcessor(num_top_types=9)

itp.fit(train_X_df).transform(train_X_df).type.value_counts()

Single Family Residence                            5709
Condominium                                        2399
MISCELLANEOUS                                       326
Multi-Family Dwellings                              230
Contact Agent                                        70
Miscellaneous Structures - Ranch, Farm Fixtures      10
Others                                                7
Residential - Vacant Land                             5
Cooperative                                           4
Miscellaneous (Residential)                           3
Name: type, dtype: int64

In [109]:
# Class YearBuiltProcessor dùng để xử lý cột info_year_built.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class YearBuiltProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, year_to_period=True):
        self.year_to_period = year_to_period

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        transformed_df = X_df
        
        # Lấy cột năm xây dựng ra.
        year_built = transformed_df.info_year_built
        time_built = year_built.astype(str)
        
        # Xóa đi các cột không cần thiết.
        transformed_df = X_df.drop(["info_year_built"], axis=1)
        
        # Chuyển từ year sang khoảng thời gian.
        if (self.year_to_period == True):
            for i in year_built.index:
                year = year_built[i]
                if year < 1900:
                    time_built[i] = "Before1900"
                if year >= 1900 and year < 1950:
                    time_built[i] = "From1900ToBefore1950"
                if year >= 1950 and year < 2000:
                    time_built[i] = "From1950ToBefore2000"
                if year >= 2000:
                    time_built[i] = "After2000"

        
        # Thêm cột time_built vào DataFrame.
        transformed_df["time_built"] = time_built
        return transformed_df

In [110]:
# TEST YearBuiltProcessor
itp = YearBuiltProcessor(True)

itp.fit(train_X_df).transform(train_X_df).time_built.value_counts()

From1950ToBefore2000    6883
After2000                879
nan                      664
From1900ToBefore1950     335
Before1900                 2
Name: time_built, dtype: int64

In [111]:
# TEST YearBuiltProcessor
itp = YearBuiltProcessor(False)

itp.fit(train_X_df).transform(train_X_df).time_built.value_counts()

nan       664
1964.0    252
1989.0    250
1963.0    225
1972.0    216
1977.0    210
1973.0    205
1965.0    202
1976.0    199
1990.0    197
1955.0    196
1974.0    196
1971.0    186
1986.0    176
1969.0    172
1980.0    169
1978.0    166
1979.0    165
1983.0    162
1987.0    161
1962.0    161
1956.0    158
1984.0    147
1959.0    144
1954.0    144
1985.0    142
1975.0    138
1968.0    127
1960.0    122
2003.0    119
         ... 
1937.0      8
1926.0      7
1936.0      7
1921.0      7
1940.0      7
1941.0      6
1939.0      5
1944.0      4
1915.0      4
1931.0      4
1942.0      4
1914.0      4
1933.0      3
1932.0      3
1912.0      3
1910.0      3
1916.0      2
1904.0      2
1934.0      2
1908.0      2
1907.0      2
1917.0      2
1943.0      1
1899.0      1
1911.0      1
1918.0      1
1919.0      1
1913.0      1
1900.0      1
1894.0      1
Name: time_built, Length: 112, dtype: int64

In [112]:
# Class TaxesProcessor dùng để xử lý các cột taxes_total và taxes_taxes.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.

class TaxesProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        # Extract ra tiền thuế phải trả.
        taxes = X_df.taxes_taxes.str.extract(r'(\d+)', expand=False)
        
        # Xóa đi các cột không cần thiết.
        transformed_df = X_df.drop(["taxes_total", "taxes_taxes"], axis=1)
        
        # Thêm cột taxes_taxes với giá trị mới extract được vào.
        transformed_df["taxes_taxes"] = taxes
        return transformed_df

In [113]:
# TEST InfoTypeProcessor
tp = TaxesProcessor()

tp.fit(train_X_df).transform(train_X_df).taxes_taxes.value_counts()

10555    16
7259      8
5026      7
1014      6
6076      6
4875      6
4127      6
4973      6
5753      5
2749      5
1310      5
3634      5
3688      5
4986      5
6540      5
3759      5
3011      5
2669      5
2545      5
7190      5
3606      5
1152      5
6029      5
6791      4
5901      4
1244      4
1034      4
1029      4
7286      4
4356      4
         ..
1440      1
12584     1
21207     1
3920      1
7370      1
1335      1
2022      1
848       1
7377      1
3031      1
31087     1
957       1
6348      1
8207      1
4877      1
16987     1
9759      1
35916     1
2577      1
6329      1
5485      1
12127     1
6667      1
4247      1
11631     1
12704     1
8443      1
6661      1
4960      1
2488      1
Name: taxes_taxes, Length: 6348, dtype: int64

In [114]:
# Class ColumnDropper dùng để xóa các cột info_property_id, info_county, info_parcel_number và mortgage.
# Được kế thừa từ 2 class của Sklearn là BaseEstimator và TransformerMixin. 
# Việc kế thừa này giúp class của ta tự động có các phương thức như set_params, get_params, fit_transform.
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X_df, y=None):
        return self

    def transform(self, X_df, y=None):
        return X_df.drop(["info_property_id", "info_county", "info_parcel_number", "mortgage"], axis=1)

In [115]:
# TEST InfoTypeProcessor
cd = ColumnDropper()

cd.fit(train_X_df).transform(train_X_df).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 10747 to 2732
Data columns (total 21 columns):
address_street        8736 non-null object
address_locality      8763 non-null object
address_region        8763 non-null object
address_code          8763 non-null int64
date_sold             8763 non-null object
info_type             8763 non-null object
info_bedrooms         7951 non-null float64
info_bathrooms        7956 non-null float64
info_size             8361 non-null float64
info_lot_size         5931 non-null float64
info_year_built       8099 non-null float64
taxes_land            8632 non-null float64
taxes_improvements    8632 non-null float64
taxes_total           8632 non-null float64
taxes_taxes           8632 non-null object
school                8763 non-null int64
total_crime           8498 non-null float64
violent_crime         8498 non-null float64
property_crime        8498 non-null float64
foreclosures          8763 non-null int64
year_sold            

Bây giờ tạo một `pipeline` của bước tiền xử lý dữ liệu để gom tất cả các bước xử lý ở trên, thêm thao tác điền các giá trị thiếu như mô tả và tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn (dùng `StandardScaler` trong Sklearn).

In [193]:
knn_cols = ["info_size", "info_lot_size"]
most_frequent_cols = ["info_bedrooms", "info_bathrooms"]
mean_cols = ["taxes_land", "taxes_improvements", "taxes_taxes"]
median_cols = ["school", "foreclosures", "total_crime", "violent_crime", "property_crime"]
cate_cols = ["address_code", "type", "time_built", "time_sold"]

cate_pipeline = Pipeline([("most_frequent", SimpleImputer(strategy="most_frequent")),
                          ("onehot", OneHotEncoder(handle_unknown='ignore'))])


column_transformer = ColumnTransformer([("knn", KNNImputer(), knn_cols),
                                        ("most_frequent", SimpleImputer(strategy="most_frequent"), most_frequent_cols),
                                        ("mean", SimpleImputer(strategy="mean"), mean_cols),
                                        ("median", SimpleImputer(strategy="median"), median_cols),
                                        ("cate", cate_pipeline, cate_cols)])


preprocess_pipeline = Pipeline([("address", AddressProcessor()),
                                ("date_sold", DateSoldProcessor()),
                                ("info_type", InfoTypeProcessor()),
                                ("year_built", YearBuiltProcessor()),
                                ("col_dropper", ColumnDropper()),
                                ("taxes", TaxesProcessor()),
                                ("column_transformer", column_transformer),
                                ("std_scaler", StandardScaler(with_mean=False))])

In [117]:
# TEST preprocess_pipeline
preprocess_pipeline.fit_transform(train_X_df)

<8763x114 sparse matrix of type '<class 'numpy.float64'>'
	with 139296 stored elements in Compressed Sparse Row format>

# 5. Mô hình hóa dữ liệu

Giờ ta có các siêu tham số:
- `month_to_season` của class `DateSoldProcessor()`, có 2 giá trị :
   + `True`: yêu cầu chuyển từ tháng bán nhà sang bán nhà theo bốn mùa.
   + `False`: để nguyên các tháng bán nhà.
   + Giá trị mặc định lúc code là `True`.
- `num_top_types` của class `InfoTypeProcessor()`, giá trị là số nguyên dương. Trong tập train có khoảng 13 loại type. Giờ mình chỉ thử tập giá trị sau [1, 3, 5, 7, 9, 11, 13]. Giá trị mặc định lúc code là `5`.
- `year_to_period` của class `YearBuiltProcessor()`, có 2 giá trị :
   + `True`: yêu cầu chuyển từ năm xây dựng nhà sang các khoảng thời gian xây nhà .
   + `False`: để nguyên các năm xây nhà.
   + Giá trị mặc định lúc code là `True`.
   
Vậy ta muốn thử tất cả các giá trị ở trên thì tốn `3` vòng lặp với `2x7x2` là `28` lần chạy.

## 5.1. Linear Regression

Giờ ta thử với mô hình `Linear Regression` với đủ 3 bộ tham số trên

In [194]:
full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", LinearRegression())])

In [195]:
month_to_season_s = [True, False]
year_to_period_s = [True, False]
num_top_types_s = [1, 3, 5, 7, 9, 11, 13]

best_val_err_LR = float('inf')
best_month_to_season_LR = True
best_year_to_period_LR = True
best_num_top_types_LR = 0

train_errs = []
val_errs = []

for month_to_season in month_to_season_s:
    for year_to_period in year_to_period_s:
        for num_top_types in num_top_types_s:
            # Set các tham số mới cho full_pipeline
            full_pipeline.set_params(preprocess__date_sold__month_to_season=month_to_season,
                                     preprocess__year_built__year_to_period=year_to_period,
                                     preprocess__info_type__num_top_types=num_top_types)
            
            # Fit full_pipeline vào tập huấn luyện
            full_pipeline.fit(train_X_df, train_y_sr)
    
            # Độ lỗi trên tập huấn luyện
            train_err = 100 - full_pipeline.score(train_X_df, train_y_sr)*100
    
            # Độ lỗi tên tập validation
            val_err = 100 - full_pipeline.score(val_X_df, val_y_sr)*100
        
            # Lưu lại các giá trị độ lỗi
            train_errs.append(train_err)
            val_errs.append(val_err)
        
            # Lưu lại best_val_err, best_alpha, best_num_top_titles
            if (val_err < best_val_err_LR): 
                best_val_err_LR = val_err
                best_month_to_season_LR = month_to_season
                best_year_to_period_LR = year_to_period
                best_num_top_types_LR = num_top_types
            
            print("With month_to_season = " + str(month_to_season) + ", year_to_period = " + str(year_to_period) + " and num_top_types = " + str(num_top_types) + (":"))
            print(" + train_err: " + str(train_err))
            print(" + train_err: " + str(val_err))

With month_to_season = True, year_to_period = True and num_top_types = 1:
 + train_err: 76.30336150055886
 + train_err: 63.19642528441863
With month_to_season = True, year_to_period = True and num_top_types = 3:
 + train_err: 73.19519307812502
 + train_err: 60.91786611163863
With month_to_season = True, year_to_period = True and num_top_types = 5:
 + train_err: 72.59982377413813
 + train_err: 60.81209462939404
With month_to_season = True, year_to_period = True and num_top_types = 7:
 + train_err: 70.94568352569252
 + train_err: 59.42398143194044
With month_to_season = True, year_to_period = True and num_top_types = 9:
 + train_err: 70.9407730543621
 + train_err: 59.40595387313977
With month_to_season = True, year_to_period = True and num_top_types = 11:
 + train_err: 70.9396428302866
 + train_err: 59.40439029226361
With month_to_season = True, year_to_period = True and num_top_types = 13:
 + train_err: 70.93827199537559
 + train_err: 59.413561502329074
With month_to_season = True, year

In [118]:
full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                          ("regression", LinearRegression())])
error_val_LR = 0

full_pipeline.fit(train_X_df, train_y_sr)
error_train = 100 - full_pipeline.score(train_X_df, train_y_sr)*100
error_val = 100 - full_pipeline.score(val_X_df, val_y_sr)*100

print(" + error_train: " + str(error_train))
print(" + error_validation: " + str(error_val))

error_val_LR = error_val

 + error_train: 72.59982377413813
 + error_validation: 60.81209462939404


## 5.2. K-Neighbors Regressor

In [119]:
from sklearn.neighbors import KNeighborsRegressor

neighbors = [1, 3, 5, 7, 9, 11]
best_neighbors_KNR = 0
best_error_val_KNR = float('inf')

for n in neighbors:
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                              ("regression", KNeighborsRegressor(n_neighbors=n))])

    full_pipeline.fit(train_X_df, train_y_sr)
    error_train = 100 - full_pipeline.score(train_X_df, train_y_sr)*100
    error_val = 100 - full_pipeline.score(val_X_df, val_y_sr)*100
    
    print("With n_neighbors = " + str(n))
    print(" + error_train: " + str(error_train))
    print(" + error_validation: " + str(error_val))
    
    if error_val < best_error_val_KNR:
        best_error_val_KNR = error_val
        best_neighbors_KNR = n

With n_neighbors = 1
 + error_train: 0.0063873032629828685
 + error_validation: 19.273275573141476
With n_neighbors = 3
 + error_train: 9.216409482344247
 + error_validation: 22.390799449197985
With n_neighbors = 5
 + error_train: 15.746979819901469
 + error_validation: 24.075030658647194
With n_neighbors = 7
 + error_train: 20.548737920453206
 + error_validation: 27.83458256385201
With n_neighbors = 9
 + error_train: 24.798967675236
 + error_validation: 35.51147013092999
With n_neighbors = 11
 + error_train: 29.62675159815747
 + error_validation: 41.96661525705745


In [55]:
print(best_error_val_KNR)
print(best_neighbors_KNR)

20.353580744925836
1


## 5.3. Decision Tree Regressor

In [120]:
from sklearn.tree import DecisionTreeRegressor

best_depth_DTR = 0
best_error_val_DTR = float('inf')

for depth in range(3,21):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                              ("regression", DecisionTreeRegressor(random_state=0, max_depth = depth))])

    full_pipeline.fit(train_X_df, train_y_sr)
    error_train = 100 - full_pipeline.score(train_X_df, train_y_sr)*100
    error_val = 100 - full_pipeline.score(val_X_df, val_y_sr)*100
    
    print("With depth = " + str(depth))
    print(" + error_train: " + str(error_train))
    print(" + error_validation: " + str(error_val))
    
    if error_val < best_error_val_DTR:
        best_error_val_DTR = error_val
        best_depth_DTR = depth

With depth = 3
 + error_train: 12.406133612238222
 + error_validation: 23.203666054237942
With depth = 4
 + error_train: 9.00411887365793
 + error_validation: 9.028294568910724
With depth = 5
 + error_train: 6.4189425491449725
 + error_validation: 9.99414834080676
With depth = 6
 + error_train: 5.02856337636311
 + error_validation: 9.008873875653336
With depth = 7
 + error_train: 4.2441991986717085
 + error_validation: 9.567176508519736
With depth = 8
 + error_train: 3.132866357860081
 + error_validation: 20.238078207975022
With depth = 9
 + error_train: 1.6403510936901
 + error_validation: 5.472689223182272
With depth = 10
 + error_train: 0.46246347595740644
 + error_validation: 8.66620489072983
With depth = 11
 + error_train: 0.26130094845004237
 + error_validation: 7.410907835735188
With depth = 12
 + error_train: 0.13000659462885267
 + error_validation: 9.475003520655761
With depth = 13
 + error_train: 0.08074241766722423
 + error_validation: 21.290496243095276
With depth = 14
 + e

In [59]:
print(best_error_val_DTR)
print(best_depth_DTR)

16.51533105213325
8


## 5.4. Random Forest Regressor

In [60]:
from sklearn.ensemble import RandomForestRegressor

best_depth_RFR = 0
best_error_val_RFR = float('inf')

for depth in range(3,21):
    full_pipeline = Pipeline([("preprocess", preprocess_pipeline),
                              ("regression", RandomForestRegressor(n_estimators=10, random_state=0, max_depth=depth))])

    full_pipeline.fit(train_X_df, train_y_sr)
    error_train = 100 - full_pipeline.score(train_X_df, train_y_sr)*100
    error_val = 100 - full_pipeline.score(val_X_df, val_y_sr)*100
    
    print("With depth = " + str(depth))
    print(" + error_train: " + str(error_train))
    print(" + error_validation: " + str(error_val))
    
    if error_val < best_error_val_RFR:
        best_error_val_RFR = error_val
        best_depth_RFR = depth

With depth = 3
 + error_train: 27.357733951800313
 + error_validation: 33.40560567890043
With depth = 4
 + error_train: 11.395160154922834
 + error_validation: 22.20369392903494
With depth = 5
 + error_train: 7.040905526446693
 + error_validation: 16.20691294342295
With depth = 6
 + error_train: 5.401936889743624
 + error_validation: 18.948915126942254
With depth = 7
 + error_train: 4.219666164873033
 + error_validation: 17.025089033221434
With depth = 8
 + error_train: 3.4165800096000822
 + error_validation: 18.97828583111456
With depth = 9
 + error_train: 3.4613522661269513
 + error_validation: 18.182043658340646
With depth = 10
 + error_train: 3.0970170426747785
 + error_validation: 12.85855351482111
With depth = 11
 + error_train: 3.257622311784843
 + error_validation: 19.545079158780965
With depth = 12
 + error_train: 3.738245539402598
 + error_validation: 18.622801473309607
With depth = 13
 + error_train: 3.1263995582746844
 + error_validation: 15.961045558380377
With depth = 14


In [61]:
print(best_error_val_RFR)
print(best_depth_RFR)

12.37721216271919
18
